# Combine population & demographic data
---

#### _Documentation in separate file_

In [1]:
import pandas as pd, numpy as np
import df_util # This loads the pandas added methods
from df_util import head, separate_by
input_path = lambda name: f'../input-data/{name}.csv'
working_path = lambda name: f'../working-data/{name}.csv'

INDEX = ['year', 'county']

---
---
---

## Census Field Descriptions
---
- To supplement the "Census Counties ..." datasets, they've provided us a table with descriptions of each column name, for each historical standard of the census. Fortunately, the 2019 and 2012 census data (that's what we're using) uses the same standard: `acs_standard`
- This script does the following:
  - Filters the source dataframe to only include `acs_standard` column descriptions
  - Selects only necessary columns (column name, description)
  - Renames some values in column name, and removes some column name values we'll never use (geonum, geojson)

In [2]:
desc = pd.read_csv(input_path('census_field_desc__qten-sdpn'))
desc = (desc
    [
        (desc.type == 'acs_standard') &
        ( ~ desc.apifieldname.isin(['geonum', 'geojson']))
    ]
    [['apifieldname', 'description']]
    .rename(columns={'apifieldname':'column'})
)
desc.iat[0,0] = 'county'
desc.to_csv(working_path('census_field_descriptions'), index=False)
desc

,column,description
0,county,Geographic Area common name
3,pop,Population Estimate for the given time range
4,hispanic,Estimate for the Hispanic Population
5,white_nh,"Estimate for the White, Non-Hispanic Population"
6,black_nh,"Estimate for the Black, Non-Hispanic Population"
...,...,...
153,emp,Estimated number of people in the civilian lab...
154,unemp,Estimated number of people in the civilian lab...
155,armedfrcs,Estimated number of people in the armed forces
156,not_lf,Estimated number of people not in the labor force


---
---
---

## Load data

In [3]:
# Population
pop_raw = pd.read_csv(working_path('county_population'))
pop_raw.year = pop_raw.year.astype(int)
head(pop_raw)

11 cols x 3904 rows


,year,county,total,male,female,over18,under19,under19_male,under19_female,over18_male,over18_female
0,1990,ADAMS,265709.0,131902.0,133807.0,184665.0,81044.0,41519.0,39525.0,90383.0,94282.0
1,1990,ALAMOSA,13617.0,6677.0,6940.0,9311.0,4306.0,2189.0,2117.0,4488.0,4823.0
2,1990,ARAPAHOE,393289.0,191722.0,201567.0,281301.0,111988.0,57241.0,54747.0,134481.0,146820.0


In [4]:
census_county_file_ids = {
    '2012':'xwky-bmsn',
    '2013':'xymp-u28i',
    '2014':'f8ak-7nmp',
    '2015':'5yyk-mqmn',
    '2016':'sn6p-34bq',
    '2017':'ewkj-ipn7',
    '2018':'xum2-smvh',
    '2019':'8j3i-rjn4',
}
# Each census year comes in a separate dataset
dem = (
    pd.concat([
            (pd.read_csv(input_path(f'census_counties_{year}__{file_id}'))
                .drop(columns=['pop', 'geonum', 'the_geom'])
                .assign(year=year)
                .rename(columns={'civ_ni_':'civ_ni_p'})
            )
        for year, file_id in census_county_file_ids.items()
    ])
    .copy()
    .rename(columns={'geoname':'county'})
    .move_col('year', 0)
)

dem.county = (dem.county
    .str.replace(" County, Colorado", "")
    .str.upper()
)
dem.year = dem.year.astype(int)
dem

,year,county,hispanic,white_nh,black_nh,ntvam_nh,asian_nh,hawpi_nh,other_nh,twoplus_nh,...,civ_ni_pop,disabled,pop16_pls,laborforce,civ_lf,emp,unemp,armedfrcs,not_lf,civ_ni_p
0,2012,ARAPAHOE,105174.0,364766.0,55629.0,2211.0,28067.0,1166.0,1267.0,16077.0,...,568663.0,49870.0,444215.0,320199.0,318041.0,292089.0,25952.0,2158.0,124016.0,568663.0
1,2012,MINERAL,15.0,671.0,9.0,5.0,1.0,0.0,0.0,1.0,...,702.0,129.0,681.0,391.0,391.0,370.0,21.0,0.0,290.0,702.0
2,2012,MONTROSE,8037.0,31799.0,186.0,74.0,227.0,49.0,33.0,589.0,...,40552.0,5649.0,32334.0,20137.0,20124.0,18110.0,2014.0,13.0,12197.0,40552.0
3,2012,PARK,777.0,14818.0,5.0,89.0,137.0,24.0,0.0,318.0,...,16042.0,1420.0,13531.0,9599.0,9583.0,8796.0,787.0,16.0,3932.0,16042.0
4,2012,MORGAN,9557.0,17399.0,701.0,153.0,44.0,0.0,0.0,346.0,...,27544.0,2946.0,21273.0,13786.0,13786.0,12758.0,1028.0,0.0,7487.0,27544.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2019,CONEJOS,4263.0,3622.0,40.0,72.0,13.0,27.0,20.0,71.0,...,8089.0,1642.0,6216.0,3383.0,3383.0,3029.0,354.0,0.0,2833.0,8089.0
60,2019,ADAMS,201784.0,252170.0,16139.0,2274.0,19493.0,536.0,722.0,10990.0,...,500418.0,52006.0,382633.0,273313.0,272810.0,261893.0,10917.0,503.0,109320.0,500418.0
61,2019,EAGLE,16179.0,36748.0,615.0,110.0,357.0,34.0,0.0,638.0,...,54502.0,2740.0,44064.0,34862.0,34775.0,34128.0,647.0,87.0,9202.0,54502.0
62,2019,MOFFAT,2044.0,10543.0,130.0,71.0,18.0,0.0,37.0,284.0,...,12968.0,1466.0,10130.0,6480.0,6445.0,6163.0,282.0,35.0,3650.0,12968.0


## Combine population data and census data
- Looking in the original demographic data, most population groups are present: gender, age, etc. So why not use those?
- A couple reasons.
  - The population dataset is likely more accurate, claiming to provide "actual" numbers, whereas the census data provides "estimates"
  - The population dataset is more precise, with age groups of each individual age number, allowing us to make our own aggregated bins (adult, minor). The census data has age groups defined already, but in increments of 5, so the middle group is "15 to 19", but we need 18 and under!
  - The population dataset offers sub-aggregations: we have `minor_female` and `minor_male`, for instance, whereas the census data only offers age populations and gender populations separately
- So instead, we will use population dataset first, and add in additional groups from census data
---

#### Select desired columns from census data

In [5]:
df = dem.copy()[['year', 'county', 'med_age',
    'households', 'avghhsize',
    'civ_lf', 'emp', 'unemp',
    'hispanic', 'white_nh', 'black_nh', 'asian_nh', 'ntvam_nh', 'hawpi_nh', 'other_nh', 'twoplus_nh',
    'pop25plus', 'hsgrad_sc',
    'med_hh_inc', 'per_cap_in',
    'citz_birth', 'citz_nat', 'born_in_co',
    'pop_3pl', 'enrolled', 'undergrad',
    'gr_1_4', 'gr_5_8', 'gr_9_12',
    'med_hm_val', 'med_yr_blt',
    'housing_un', 'occ_hu',
    'own_occ_hu', 'v_l_50k', 'v50k_100k', 'v100k_150k', 'v150k_200k', 'v200k_250k', 'v250k_300k',
    'v300k_400k', 'v400k_500k', 'v500k_750k', 'v750k_1m', 'v_1m_plus',
    'b2000_2009', 'b1990_1999', 'b1980_1989', 'b1970_1979',
    'b1960_1969', 'b1950_1959', 'b1940_1949', 'b1939_e',
    'ps_uni', 'ps_below',
    'tot_l18', 'pov_l18',
]]

#### Group bins together

In [6]:
# Create new variable for total citizens. Place it next to citz_birth
df = (df
    .insert_at('citz', 'citz_birth', df.citz_birth + df.citz_nat)
    .drop(columns='citz_nat')
    .combine_cols(items={
        'race_other': ['ntvam_nh', 'hawpi_nh', 'other_nh', 'twoplus_nh'],
        'b1949_e': ['b1939_e', 'b1940_1949'],
        'v50k_150k':  ['v50k_100k', 'v100k_150k'],
        'v150k_250k': ['v150k_200k', 'v200k_250k'],
        'v250k_400k': ['v250k_300k', 'v300k_400k'],
        'v400k_750k': ['v400k_500k', 'v500k_750k'],
        'v750k_plus': ['v750k_1m', 'v_1m_plus'],
    })
)
df

,year,county,med_age,households,avghhsize,civ_lf,emp,unemp,hispanic,white_nh,...,b1990_1999,b1980_1989,b1970_1979,b1960_1969,b1950_1959,b1949_e,ps_uni,ps_below,tot_l18,pov_l18
0,2012,ARAPAHOE,35.7,223747.0,2.55,318041.0,292089.0,25952.0,105174.0,364766.0,...,33989.0,56011.0,62253.0,22258.0,16519.0,7165.0,568999.0,66945.0,144576.0,23054.0
1,2012,MINERAL,60.3,363.0,1.83,391.0,370.0,21.0,15.0,671.0,...,232.0,239.0,203.0,100.0,75.0,240.0,702.0,47.0,26.0,0.0
2,2012,MONTROSE,42.6,16732.0,2.41,20124.0,18110.0,2014.0,8037.0,31799.0,...,3750.0,2106.0,3581.0,1298.0,920.0,2333.0,40368.0,5565.0,9788.0,1927.0
3,2012,PARK,47.0,6997.0,2.29,9583.0,8796.0,787.0,777.0,14818.0,...,3567.0,2374.0,2952.0,1051.0,693.0,939.0,16049.0,1355.0,3049.0,276.0
4,2012,MORGAN,36.0,10489.0,2.62,13786.0,12758.0,1028.0,9557.0,17399.0,...,1195.0,984.0,2282.0,1078.0,1858.0,2703.0,27416.0,4002.0,7670.0,1454.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2019,CONEJOS,38.8,3183.0,2.53,3383.0,3029.0,354.0,4263.0,3622.0,...,694.0,565.0,665.0,342.0,265.0,1358.0,8089.0,1756.0,2164.0,636.0
60,2019,ADAMS,33.8,166450.0,3.00,272810.0,261893.0,10917.0,201784.0,252170.0,...,28459.0,22069.0,29342.0,17026.0,19373.0,6023.0,499315.0,54159.0,134212.0,19943.0
61,2019,EAGLE,37.0,18171.0,3.00,34775.0,34128.0,647.0,16179.0,36748.0,...,9377.0,7555.0,6022.0,907.0,318.0,934.0,54401.0,4354.0,11805.0,1184.0
62,2019,MOFFAT,36.6,5366.0,2.42,6445.0,6163.0,282.0,2044.0,10543.0,...,804.0,751.0,2168.0,567.0,472.0,761.0,13003.0,2206.0,3361.0,680.0


#### Create nominal variables for housing price and housing age
- First, create a categorical variable whose values are the COLUMN NAME of the bin with the max value. For instance, if a given county has more houses in the `v50k_100k` range than any other range, the value at that row in the new column will be "v50k_100k"
- Next, create a nominal column from that categorical column, ordered so that a lower number means less desirable. So for prices, "v_l_50k" -> 1, and for year built, "b1939_e" -> 1

In [7]:
blt_ascending = ['b1949_e','b1950_1959','b1960_1969','b1970_1979','b1980_1989','b1990_1999','b2000_2009']
prices_ascending = ['v_l_50k', 'v50k_150k', 'v150k_250k', 'v250k_400k', 'v400k_750k', 'v750k_plus']
df = (df
    .add_binmax('blt_freq_yr', blt_ascending)
    .add_ordinal('blt_freq_yr', blt_ascending)
    .add_binmax('hu_freq_val', prices_ascending)
    .add_ordinal('hu_freq_val', prices_ascending)
)

---

#### Rename everything, with a naming system that let's us easily select sub-groups of columns with a simple string match

In [8]:
# If you're wondering why we're doing all this renaming, look at the
# beginning of each new name. Notice a pattern?
pop = pop_raw.rename(columns={
    'total':            'pop',

    'male':             'gend_m',
    'female':           'gend_f',

    'over18':           'age_over18',
    'under19':          'age_undr19',

    'over18_male':      'gend_m_age_over18',
    'over18_female':    'gend_f_age_over18',
    'under19_male':     'gend_m_age_undr19',
    'under19_female':   'gend_f_age_undr19',
})
df = df.rename(columns={
    'med_age':      'age_median',

    'per_cap_in':   'inc_per_cap',
    'med_hh_inc':   'inc_hh_median',

    'households':   'hh',
    'avghhsize':    'hh_size_avg',

    'pop25plus':    'hsgrad_pool',
    'hsgrad_sc':    'hsgrad_graduated',

    'born_in_co':   'citz_co',
    'citz_birth':   'citz_birth',

    'emp':          'civ_lf_employed',

    'hispanic':     'race_hispanic',
    'white_nh':     'race_white',
    'black_nh':     'race_black',
    'asian_nh':     'race_asian',

    'ps_uni':       'ps_known',
    'ps_below':     'ps_below',
    'tot_l18':      'ps_undr18_known',
    'pov_l18':      'ps_undr18_below',

    'pop_3pl':      'stud_enroll_pool',
    'enrolled':     'stud_enrolled',
    'undergrad':    'stud_undergrad',
    'gr_1_4':       'stud_1_4',
    'gr_5_8':       'stud_5_8',
    'gr_9_12':      'stud_9_12',

    'housing_un':   'hu',
    'occ_hu':       'hu_occ',

    'blt_freq_yr':  'hu_blt_freq_yr',
    'blt_freq_yr_ord':'hu_blt_freq_yr_ord',
    'b1949_e':      'hu_blt_lt_1950',
    'b1950_1959':   'hu_blt_1950_1959',
    'b1960_1969':   'hu_blt_1960_1969',
    'b1970_1979':   'hu_blt_1970_1979',
    'b1980_1989':   'hu_blt_1980_1989',
    'b1990_1999':   'hu_blt_1990_1999',
    'b2000_2009':   'hu_blt_2000_plus',

    'own_occ_hu':   'hu_oo',
    'hu_freq_val':  'hu_oo_freq_val',
    'hu_freq_val_ord':'hu_oo_freq_val_ord',
    'v_l_50k':      'hu_oo_lt_50',
    'v50k_150k':    'hu_oo_50_150',
    'v150k_250k':   'hu_oo_150_250',
    'v250k_400k':   'hu_oo_250_400',
    'v400k_750k':   'hu_oo_400_750',
    'v750k_plus':   'hu_oo_750_plus',
})

## Merge population and census data

In [9]:
main = (pop
    .merge(df, on=['year', 'county'])
    .move_col('age_median', 'age_over18')
)
head(main)

61 cols x 512 rows


,year,county,pop,gend_m,gend_f,age_median,age_over18,age_undr19,gend_m_age_undr19,gend_f_age_undr19,...,hu_blt_1970_1979,hu_blt_1960_1969,hu_blt_1950_1959,hu_blt_freq_yr_ord,hu_blt_freq_yr,hu_blt_lt_1950,ps_known,ps_below,ps_undr18_known,ps_undr18_below
0,2012,ADAMS,460468.0,231571.0,228902.0,32.4,324757.0,135711.0,69462.0,66249.0,...,30185.0,19615.0,20369.0,7,b2000_2009,6158.0,438171.0,62008.0,124375.0,25278.0
1,2012,ALAMOSA,15606.0,7823.0,7792.0,32.2,11367.0,4239.0,2201.0,2044.0,...,1405.0,654.0,591.0,1,b1949_e,1536.0,14622.0,3191.0,3817.0,758.0
2,2012,ARAPAHOE,596226.0,292548.0,303673.0,35.7,439465.0,156761.0,80341.0,76419.0,...,62253.0,22258.0,16519.0,4,b1970_1979,7165.0,568999.0,66945.0,144576.0,23054.0


## Calculations for groups
---

In [10]:
from grouped_df import GroupedDF
GroupedDF.default_index = INDEX
GroupedDF.set_groups(['age', 'gend', 'race', 'inc', 'hh', 'citz', 'hsgrad', 'civ_lf', 'ps', 'stud', 'hu', 'hu_blt', 'hu_oo'])

In [11]:
gd = GroupedDF(main, INDEX, custom={'hu': INDEX + ['hu', 'hu_occ']})
# gd.df
gd.display(5)

age: 


,year,county,age_median,age_over18,age_undr19
0,2012,ADAMS,32.4,324757.0,135711.0
1,2012,ALAMOSA,32.2,11367.0,4239.0
2,2012,ARAPAHOE,35.7,439465.0,156761.0
3,2012,ARCHULETA,47.5,9539.0,2450.0
4,2012,BACA,47.8,2872.0,830.0



gend: 


,year,county,gend_m,gend_f,gend_m_age_undr19,gend_f_age_undr19,gend_m_age_over18,gend_f_age_over18
0,2012,ADAMS,231571.0,228902.0,69462.0,66249.0,162109.0,162653.0
1,2012,ALAMOSA,7823.0,7792.0,2201.0,2044.0,5622.0,5748.0
2,2012,ARAPAHOE,292548.0,303673.0,80341.0,76419.0,212207.0,227254.0
3,2012,ARCHULETA,6033.0,5960.0,1297.0,1150.0,4736.0,4810.0
4,2012,BACA,1836.0,1869.0,422.0,409.0,1414.0,1460.0



race: 


...

,year,county,hu,hu_occ
0,2012,ADAMS,163245.0,151034.0
1,2012,ALAMOSA,6572.0,5853.0
2,2012,ARAPAHOE,238160.0,223747.0
3,2012,ARCHULETA,8742.0,4536.0
4,2012,BACA,2253.0,1675.0



hu_blt: 


,year,county,hu_blt_2000_plus,hu_blt_1990_1999,hu_blt_1980_1989,hu_blt_1970_1979,hu_blt_1960_1969,hu_blt_1950_1959,hu_blt_freq_yr_ord,hu_blt_freq_yr,hu_blt_lt_1950
0,2012,ADAMS,38682.0,27598.0,20368.0,30185.0,19615.0,20369.0,7,b2000_2009,6158.0
1,2012,ALAMOSA,650.0,866.0,862.0,1405.0,654.0,591.0,1,b1949_e,1536.0
2,2012,ARAPAHOE,39415.0,33989.0,56011.0,62253.0,22258.0,16519.0,4,b1970_1979,7165.0
3,2012,ARCHULETA,2204.0,2186.0,2054.0,1384.0,326.0,124.0,7,b2000_2009,415.0
4,2012,BACA,46.0,172.0,172.0,470.0,284.0,306.0,1,b1949_e,803.0



hu_oo: 


,year,county,hu_oo,hu_oo_freq_val_ord,hu_oo_freq_val,hu_oo_lt_50,hu_oo_50_150,hu_oo_150_250,hu_oo_250_400,hu_oo_400_750,hu_oo_750_plus
0,2012,ADAMS,100108.0,3,v150k_250k,8578.0,19838.0,47583.0,17779.0,5427.0,903.0
1,2012,ALAMOSA,3702.0,2,v50k_150k,435.0,1599.0,1077.0,397.0,177.0,17.0
2,2012,ARAPAHOE,143158.0,3,v150k_250k,4207.0,22174.0,55935.0,38213.0,16339.0,6290.0
3,2012,ARCHULETA,3532.0,4,v250k_400k,152.0,513.0,781.0,1153.0,612.0,321.0
4,2012,BACA,1236.0,2,v50k_150k,399.0,601.0,144.0,49.0,23.0,20.0


## Calculations
---

- **age, and gend**
  - `age_median`: (Existing)
  - `age_undr19_prop`: What percent of the population is under 19?
  - `gend_m_prop`: What percent of the population is male?
  - `age_undr19_gend_m_prop`: What percent of under-19 year old are male? (divide m_undr19 by undr19)
- **inc**
  - `inc_hh_med`: (Existing) Median household income
  - `inc_per_cap`: (Existing) Per capita income
- **hh**
  - `hh_size_avg`: (Existing) Average household size
- **race**
  - `race_{x}_prop`: What percent of the population is race x?
  - `race_prop_stdev`: What is the standard deviation of the race proportions? We need to calculate the proportions first, to normalize for the population size, that way, we can compare the standard deviations across groups
- **hsgrad**
  - `hsgrad_graduated_prop`: What percent of adults (age 25+) have a high school diploma or equivalent?
- **civ_lf**
  - `civ_lf_prop`: What percent of the population is in the civilian labor force?
  - `civ_lf_employed_prop`: What percent of the civilian labor force is employed?
- **ps**
  - `ps_total_prop`: What percent of people whose poverty status is known are below the poverty line?
  - `ps_undr18_total_prop`: What percent of under-18 people whose poverty status is known are below the poverty line?
  - `ps_undr18_prop`: What percent of people below the poverty line are under 18?
- **stud**
  - `stud_enrolled_prop`: Percent of people who could be enrolled in school that actually are enrolled
  - `stud_hs_prop`: What percent of gradeschool students (1-12) are high schoolers? (lower number indicates dropouts, which may associate with crime)
  - `stud_undergrad_prop`: What percent of enrolled students are undergraduates?
- **citz**
  - `citz_prop`: What percent of the population is a us citizen?
  - `citz_birth_prop`: What percent of us citizens were born in the us?
  - `citz_co_prop`: What percent of citizens were born in Colorado?
- **hu**
  - `hu_occ_prop`: Percent of homes which are occupied
  - `hu_blt_after1989`: Percent of homes which were built in the past 20 years
  - `hu_blt_nominal`: Convert hu_blt_mode_range to nominal, where the highest number corresponds to highest year range
- **hu_oo**
  - `hu_oo_prop`: Percent of occupied properties occupied by owner. The remaining percent is renter occupied
  - `hu_oo_lt_50_prop`: Percent of owner occupied properties worth less than $50,000
  - `hu_oo_750_plus_prop`: Percent of owner occupied properties worth $750,000 or more


In [12]:
df = main.copy()

df['age_over18_prop'] = df.age_over18 / df['pop']
df['age_undr19_prop'] = df.age_undr19 / df['pop']
df['gend_m_prop'] = df.gend_m / df['pop']
df['gend_f_prop'] = df.gend_f / df['pop']
df['age_undr19_gend_m_prop'] = df.gend_m_age_undr19 / df.age_undr19
df['age_undr19_gend_f_prop'] = df.gend_f_age_undr19 / df.age_undr19
df['age_over18_gend_m_prop'] = df.gend_m_age_over18 / df.age_over18
df['age_over18_gend_f_prop'] = df.gend_f_age_over18 / df.age_over18

df['gend_m_age_undr19_prop'] = df.gend_m_age_undr19 / df.gend_m
df['gend_m_age_over18_prop'] = df.gend_m_age_over18 / df.gend_m
df['gend_f_age_undr19_prop'] = df.gend_f_age_undr19 / df.gend_f
df['gend_f_age_over18_prop'] = df.gend_f_age_over18 / df.gend_f

race_base = GroupedDF(df, INDEX).race
race = df.copy()[INDEX]
for c in [c for c in race_base.columns if c not in INDEX]:
    race[f'{c}_prop'] = race_base[c] / df['pop']

race['race_prop_stdev'] = np.std(race.drop(columns=INDEX), axis=1)
df = df.merge(race, how='inner', on=INDEX)

df['hsgrad_graduated_prop'] = df.hsgrad_graduated / df.hsgrad_pool

df['civ_lf_prop'] = df.civ_lf / df['pop']
df['civ_lf_employed_prop'] = df.civ_lf_employed / df.civ_lf

df['ps_total_prop'] = df.ps_below / df.ps_known
df['ps_undr18_total_prop'] = df.ps_undr18_below / df.ps_undr18_known
df['ps_undr18_prop'] = df.ps_undr18_below / df.ps_below

df['stud_enrolled_prop'] = df.stud_enrolled / df.stud_enroll_pool
df['stud_hs_prop'] = df.stud_9_12 / (df.stud_1_4 + df.stud_5_8 + df.stud_9_12)
df['stud_undergrad_prop'] = df.stud_undergrad / df.stud_enrolled

df['citz_per_cap'] = df.citz / df['pop']
df['citz_birth_prop'] = df.citz_birth / df.citz
df['citz_co_prop'] = df.citz_co / df.citz

df['hu_per_cap'] = df.hu / df['pop']
df['hu_occ_prop'] = df.hu_occ / df.hu
df['hu_blt_2000_plus_prop'] = df.hu_blt_2000_plus / df.hu

df['hu_oo_prop'] = df.hu_oo / df.hu_occ

for hval in ['hu_oo_lt_50', 'hu_oo_50_150', 'hu_oo_150_250', 'hu_oo_250_400', 'hu_oo_400_750', 'hu_oo_750_plus']:
    df[f'{hval}_prop'] = df[hval] / df.hu_oo

for hyear in [
        'hu_blt_lt_1950', 'hu_blt_1950_1959', 'hu_blt_1960_1969',
        'hu_blt_1970_1979', 'hu_blt_1980_1989', 'hu_blt_1990_1999', 'hu_blt_2000_plus'
    ]:
    df[f'{hyear}_prop'] = df[hyear] / df.hu

prop, counts = separate_by(df, ['prop', 'per_cap', 'median', 'avg', 'freq', 'med_hm_val', 'med_yr_blt'], index=INDEX)


prop.to_csv(working_path('county_stats_normalized'), index=False)
counts.to_csv(working_path('county_stats_counts'), index=False)
df.to_csv(working_path('county_stats'), index=False)

gprop = GroupedDF(prop, INDEX, custom={'hu': INDEX + ['hu_per_cap', 'hu_occ_prop']})
gprop.display()

age: 


,year,county,age_over18_prop,age_undr19_prop,age_undr19_gend_m_prop,age_undr19_gend_f_prop,age_over18_gend_m_prop,age_over18_gend_f_prop,age_median
0,2012,ADAMS,0.705276,0.294724,0.511838,0.488162,0.499170,0.500845,32.4
1,2012,ALAMOSA,0.728374,0.271626,0.519226,0.482189,0.494590,0.505674,32.2
2,2012,ARAPAHOE,0.737078,0.262922,0.512506,0.487487,0.482876,0.517115,35.7



gend: 


,year,county,gend_m_prop,gend_f_prop,gend_m_age_undr19_prop,gend_m_age_over18_prop,gend_f_age_undr19_prop,gend_f_age_over18_prop
0,2012,ADAMS,0.502904,0.497107,0.299960,0.700040,0.289421,0.710579
1,2012,ALAMOSA,0.501282,0.499295,0.281350,0.718650,0.262320,0.737680
2,2012,ARAPAHOE,0.490666,0.509325,0.274625,0.725375,0.251649,0.748351



race: 


...

,year,county,hu_per_cap,hu_occ_prop
0,2012,ADAMS,0.354520,0.925198
1,2012,ALAMOSA,0.421120,0.890596
2,2012,ARAPAHOE,0.399446,0.939482



hu_blt: 


,year,county,hu_blt_2000_plus_prop,hu_blt_lt_1950_prop,hu_blt_1950_1959_prop,hu_blt_1960_1969_prop,hu_blt_1970_1979_prop,hu_blt_1980_1989_prop,hu_blt_1990_1999_prop,hu_blt_freq_yr_ord,hu_blt_freq_yr
0,2012,ADAMS,0.236957,0.037722,0.124776,0.120157,0.184906,0.124770,0.169059,7,b2000_2009
1,2012,ALAMOSA,0.098904,0.233719,0.089927,0.099513,0.213786,0.131163,0.131771,1,b1949_e
2,2012,ARAPAHOE,0.165498,0.030085,0.069361,0.093458,0.261392,0.235182,0.142715,4,b1970_1979



hu_oo: 


,year,county,hu_oo_prop,hu_oo_lt_50_prop,hu_oo_50_150_prop,hu_oo_150_250_prop,hu_oo_250_400_prop,hu_oo_400_750_prop,hu_oo_750_plus_prop,hu_oo_freq_val_ord,hu_oo_freq_val
0,2012,ADAMS,0.662818,0.085687,0.198166,0.475317,0.177598,0.054211,0.009020,3,v150k_250k
1,2012,ALAMOSA,0.632496,0.117504,0.431929,0.290924,0.107239,0.047812,0.004592,2,v50k_150k
2,2012,ARAPAHOE,0.639821,0.029387,0.154892,0.390722,0.266929,0.114133,0.043937,3,v150k_250k
